In [ ]:
def reverse_substitutions(sequence: str):
    """Identify and return all reversing substitutions in a single string.
    A reversing substitution is a pattern like A->B->A (original -> changed -> reverted).
    Example: reverse_substitutions('AAGGAA') -> [(1, 'A->G->A'), (2, 'A->G->A')]
    """
    results = []
    n = len(sequence)
    for i in range(1, n - 1):
        if sequence[i - 1] == sequence[i + 1] and sequence[i - 1] != sequence[i]:
            results.append((i, f"{sequence[i-1]}->{sequence[i]}->{sequence[i+1]}"))
    return results


def main():
    seq = input("Enter a sequence: ").strip().upper()
    res = reverse_substitutions(seq)
    if not res:
        print("No reversing substitutions found.")
    else:
        print("Reversing substitutions:")
        for pos, pattern in res:
            print(f"Position {pos+1}: {pattern}")


if __name__ == '__main__':
    main()

In [3]:
import re

class Node:
    def __init__(self, name=None):
        self.name = name
        self.children = []
        self.parent = None
        self.seq = None

    def __repr__(self):
        return f"Node({self.name})"


def parse_newick(s: str) -> Node:
    s = s.strip()
    if not s.endswith(';'):
        raise ValueError('Newick must end with ;')
    s = s[:-1]

    stack = []
    i = 0
    while i < len(s):
        c = s[i]
        if c == '(':
            stack.append('(')
            i += 1
        elif c == ',':
            i += 1
        elif c == ')':
            kids = []
            while stack and stack[-1] != '(':
                kids.append(stack.pop())
            stack.pop()
            kids.reverse()
            i += 1
            m = re.match(r"([A-Za-z0-9_.-]+)", s[i:])
            label = None
            if m:
                label = m.group(1)
                i += len(label)
            node = Node(label)
            for child in kids:
                child.parent = node
                node.children.append(child)
            stack.append(node)
        elif c.isspace():
            i += 1
        else:
            m = re.match(r"([A-Za-z0-9_.-]+)", s[i:])
            label = m.group(1)
            node = Node(label)
            stack.append(node)
            i += len(label)

    return stack[0]


def parse_fasta(s: str):
    records = {}
    cur = None
    for line in s.splitlines():
        line = line.strip()
        if not line:
            continue
        if line.startswith('>'):
            cur = line[1:]
            records[cur] = []
        else:
            records[cur].append(line)
    for k in list(records.keys()):
        records[k] = ''.join(records[k])
    return records


def collect_nodes(root: Node):
    res = {}
    stack = [root]
    while stack:
        u = stack.pop()
        if u.name:
            res[u.name] = u
        for c in u.children:
            stack.append(c)
    return res


def find_reversing_substitutions(root: Node):
    nodes_by_name = collect_nodes(root)
    L = len(next(iter(nodes_by_name.values())).seq)

    results = []
    for parent_node in nodes_by_name.values():
        for child in parent_node.children:
            s = parent_node.seq
            t = child.seq
            for i in range(L):
                if s[i] != t[i]:
                    a, b = s[i], t[i]
                    stack = [child]
                    while stack:
                        u = stack.pop()
                        if u.seq[i] != b:
                            continue
                        for v in u.children:
                            if v.seq[i] == a:
                                results.append((child.name, v.name, i+1, f"{a}->{b}->{a}"))
                            elif v.seq[i] == b:
                                stack.append(v)
    return results


def main():
    print("Paste your input below (Newick tree first, then FASTA). Press Ctrl+D (Linux/Mac) or Ctrl+Z then Enter (Windows) when done:\n")
    import sys
    data = sys.stdin.read()
    idx = data.find(';')
    newick = data[:idx+1].strip()
    fasta_part = data[idx+1:]

    root = parse_newick(newick)
    fasta = parse_fasta(fasta_part)
    nodes = collect_nodes(root)

    for name, node in nodes.items():
        if name not in fasta:
            raise ValueError(f"No sequence for node '{name}' in FASTA input")
        node.seq = fasta[name]

    results = find_reversing_substitutions(root)
    print("\nResults:\n")
    for t_name, w_name, pos, rep in results:
        print(t_name, w_name, pos, rep)

if __name__ == '__main__':
    main()

Paste your input below (Newick tree first, then FASTA). Press Ctrl+D (Linux/Mac) or Ctrl+Z then Enter (Windows) when done:



ValueError: Newick must end with ;